In [4]:
from food.tools import *
from food.paths import *
from food.psql import *

import pandas as pd
import numpy as np
from tqdm import tqdm
import food.custom_pandas as cpd

from food.qdrant import *

In [3]:
client.get_collections()

['food_text', 'glovo']

In [5]:
collection_name

'food_text'

In [6]:
project_name = "food"
table = 'foods'

dim = 768
limit = 100000

In [7]:
query = f"""select f.id,f.description,f.clip
        FROM      {project_name}.{table}    f
        
        WHERE f.clip    is not null"""

In [8]:
for df in tqdm(cpd.read_sql_query(query, engine, chunksize=limit), desc="qdrant_update {project_name}"):

    ids = df['id'].tolist()
    clip = df['clip']
    clip = np.array([np.array(c) for c in clip.values])

    df = df.fillna('0')


    payload = df.drop(columns = ['id','clip']).to_dict('records')

    client.upload_collection(
        collection_name=collection_name,
        vectors=clip,
        payload=payload,
        ids=ids,
        parallel=4)

qdrant_update {project_name}: 0it [00:00, ?it/s]2022-06-24 23:52:06.288 | INFO     | qdrant_client.parallel_processor:_worker:45 - Reader worker: 0 PID: 3045604
2022-06-24 23:52:06.296 | INFO     | qdrant_client.parallel_processor:_worker:45 - Reader worker: 1 PID: 3045607
2022-06-24 23:52:06.304 | INFO     | qdrant_client.parallel_processor:_worker:45 - Reader worker: 2 PID: 3045610
2022-06-24 23:52:06.312 | INFO     | qdrant_client.parallel_processor:_worker:45 - Reader worker: 3 PID: 3045613
2022-06-24 23:52:10.522 | INFO     | qdrant_client.parallel_processor:_worker:77 - Reader worker 3 finished
2022-06-24 23:52:10.566 | INFO     | qdrant_client.parallel_processor:_worker:77 - Reader worker 2 finished
2022-06-24 23:52:10.588 | INFO     | qdrant_client.parallel_processor:_worker:77 - Reader worker 1 finished
111it [00:04, 25.70it/s]
qdrant_update {project_name}: 1it [00:05,  5.95s/it]
